In [1]:
import tensorflow as tf
import numpy as np
import tsp_env
from pointer_net_tsp import *
from critic_network_tsp import *

In [ ]:
batch_size = 128; n_coords = 2; n_cities = 5; hidden_size = 128; 
embedding_size = hidden_size
# Define the network and placeholders
cities_coords_ph = tf.placeholder(tf.float32, [batch_size, n_cities, n_coords])
played_actions_ph = tf.placeholder(tf.int32, [batch_size, n_cities])
rewards_ph = tf.placeholder(tf.float32, [batch_size])
adv_ph = tf.placeholder(tf.float32, [batch_size])

# Actor network definition
loss, decoder_outputs, total_loss = pointer_network(cities_coords_ph,
                                                    played_actions_ph,
                                                    hidden_size=hidden_size,
                                                    embedding_size=embedding_size,
                                                    max_time_steps=n_cities,
                                                    input_size=n_coords,
                                                    batch_size=batch_size,
                                                    initialization_stddev=0.1)

# Critic network definition
bsln_value = critic_network(cities_coords_ph,
                            hidden_size=hidden_size,
                            embedding_size=embedding_size,
                            max_time_steps=n_cities,
                            input_size=n_coords,
                            batch_size=batch_size,
                            initialization_stddev=0.1,
                            n_processing_steps=n_cities,
                            d=embedding_size)

# Cross entropy (loss here) is the negative log likelihood of taken actions
# Rewards is negative tour length
# We want to maximize E[logprob(\tau) * reward(\tau)]
# I.e. minimize E[logprob(tau) * tour_length(\tau)]
# I.e. minimize -E[cross entropy(tau) * tour_length(\tau)]
# I.e. minimize E[cross_entropy(\tau) * reward(\tau)] = E[loss(\tau) * reward(\tau)]
pg_loss = tf.reduce_sum(loss * adv_ph)
optimizer = tf.train.AdamOptimizer(2e-3)
update_op = optimizer.minimize(pg_loss)

# Baseline loss and training op
bsln_loss = tf.losses.mean_squared_error(labels=rewards_ph,
                                         predictions=bsln_value)
bsln_train_op = tf.train.AdamOptimizer(2e-3).minimize(bsln_loss)

In [ ]:
n_baseline_gradient_steps = 10
#################################
#        POLICY GRADIENT        # 
#################################
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Iterate batch collection and gradient steps
for it in range(int(1e5)):
    # Collect batch
    envs = []
    inputs_list = []
    # Generate and initialize a batch of environments
    for i in range(batch_size):
        envs.append(tsp_env.TSP_env(n_cities, use_alternative_state=True))
        envs[-1].reset()
        inputs_list.append(envs[-1].nodes)

    inputs_batch = np.array(inputs_list)
    # Use the PointerNet on this test batch and get its predictions
    predicted_outputs = np.array(sess.run(decoder_outputs, 
                                          feed_dict={cities_coords_ph: inputs_batch})).T
    # Compute the rewards corresponding to the predicted tours
    rewards = []
    for i in range(batch_size):
        for action in predicted_outputs[i]:
            envs[i].step(action)
        rewards.append(envs[i].accumulated_reward())
        
    # Carry out baseline training steps
    for bsln_step in range(n_baseline_gradient_steps):
        bsln_loss_val, _ = sess.run([bsln_loss, bsln_train_op], 
                                    feed_dict={cities_coords_ph: inputs_batch,
                                               rewards_ph: rewards})
    
    # Compute baseline value
    bsln = sess.run(bsln_value, feed_dict={cities_coords_ph: inputs_batch})
    
    # Compute normalized advantages
    adv = np.array(rewards) - bsln
    normd_adv = (adv - np.mean(adv)) / np.std(adv)
    # Print average reward
    if it % 100 == 0:
        print('Mean reward: ', np.mean(rewards))
        print('Baseline MSE: ', bsln_loss_val)
    
    # Take a gradient step
    sess.run(update_op, feed_dict={cities_coords_ph: inputs_batch,
                                   played_actions_ph: predicted_outputs,
                                   adv_ph: normd_adv})

Mean reward:  -2.63844886844
Baseline MSE:  0.52204
Mean reward:  -2.38110814915
Baseline MSE:  0.0704803
Mean reward:  -2.39483235433
Baseline MSE:  0.0675375
Mean reward:  -2.19432494969
Baseline MSE:  0.0106988


In [ ]:
# Sanity check: supervised training
def generate_batch(n_cities, batch_size):
    inputs_list = []; labels_list = []
    env = tsp_env.TSP_env(n_cities, use_alternative_state=True)
    for i in range(batch_size):
        env.reset()
        s = env.reset()
        coords = s.reshape([4, n_cities])[:2, ].T
        inputs_list.append(coords)
        labels_list.append(env.optimal_solution()[1])
    return np.array(inputs_list), np.array(labels_list)

supervised_update_op = tf.train.AdamOptimizer(2e-3).minimize(total_loss)
# Define session, initialize variables
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Training loop
mean_approx_ratios = []
loss_vals = []
for i in range(1000):
    # import pdb; pdb.set_trace()
    inputs_batch, labels_batch = generate_batch(n_cities, batch_size)
    loss_val, _ = sess.run([total_loss, supervised_update_op], 
                         feed_dict={cities_coords_ph: inputs_batch,
                                    played_actions_ph: labels_batch})
    loss_vals.append(loss_val)
    # Test accuracy
    if i % 100 == 0:
        envs = []
        inputs_list = []
        optimal_rewards = []
        optimal_tours = []
        # Generate and initialize a batch of environments
        for i in range(batch_size):
            envs.append(tsp_env.TSP_env(n_cities, use_alternative_state=True))
            envs[-1].reset()
            inputs_list.append(envs[-1].nodes)
            optimal_solution = envs[-1].optimal_solution()
            optimal_rewards.append(optimal_solution[0])
            optimal_tours.append(optimal_solution[1])
        inputs_batch = np.array(inputs_list)
        # Use the PointerNet on this test batch and get its predictions
        predicted_outputs = np.array(sess.run(decoder_outputs, 
                                              feed_dict={cities_coords_ph: inputs_batch})).T
        # Compute the rewards corresponding to the predicted tours
        rewards = []
        for i in range(batch_size):
            for action in predicted_outputs[i]:
                envs[i].step(action)
            rewards.append(envs[i].accumulated_reward())
        # Get the approximation ratio of the predictions
        approximation_ratios = np.array(rewards) / np.array(optimal_rewards)
        mean_approx_ratios.append(np.mean(approximation_ratios))
        print(mean_approx_ratios[-1])
        print(loss_vals[-1])

In [ ]:
inputs_batch[0]

In [ ]:
labels_batch